In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!pip install pandas beautifulsoup4 konlpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 37.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 488.6/488.6 kB 40.8 MB/s eta 0:00:00


In [46]:
#패키지
import pandas as pd
import numpy as np
import re
from bs4 import BeautifulSoup
from konlpy.tag import Okt
import os

In [8]:
import nltk
from nltk.tokenize import word_tokenize

In [26]:
news=pd.read_csv("/content/drive/MyDrive/관세청공모전/크롤링데이터/news_이커머스_중국직구_알테쉬.csv")

In [27]:
news.head()

,title,raw_text,date,url
0,"""KC인증이 능사 아냐…계속 모니터링 해야""[中직구 안전성 논란④]","""KC인증이 능사 아냐…계속 모니터링 해야""[中직구 안전성 논란④] ""발암물질 및...",2024-06-03,https://www.newsis.com/view/?id=NISX20240531_0...
1,"'직구 금지' 철회한 정부, 유해물품 안전성 검증할 방법은 - 머니S",최근 정부가 해외 직구 규제 대책을 발표했다가 번복하면서 소비자 혼란이 가중되고 있...,2024-06-03,https://www.moneys.co.kr/article/2024053117523...
2,[테크왔숑] 늘어나는 해외직구에 신용카드 해킹 피해 급증···VPN으로 이중잠금,노드VPN의 다크 웹 모니터링·Double VPN 기능 통해 개인 정보 보호 철저 ...,2024-06-03,https://www.cnet.co.kr/view/?no=20240603113543
3,"짝퉁 골프, 중국계 플랫폼 중고 시장서 활개","가품 골프용품 거래 현장 [자료=클럽하우스] \n 알리, 테무, 쉬인 등 최근 급...",2024-06-04,http://jtbcgolf.joins.com/news/news_view.asp?n...
4,中 초저가 직구 급증…항공운임도 고공행진,中 초저가 직구 급증…항공운임도 고공행진 3·4분기 해상화물 몰려 더 뛸 듯 2일...,2024-06-02,https://www.hankyung.com/article/2024060299621


# 데이터 전처리

In [28]:
#한국어 불용어 리스트
stopword_file='/content/drive/MyDrive/관세청공모전/코드/stopword.txt'

In [29]:
# 불용어 리스트 파일 읽기
with open(stopword_file, 'r', encoding='utf-8') as file:
    stopwords = file.read().splitlines()

In [30]:
stopwords

['않다',
 '되어다',
 '되다',
 '하다',
 '어떻다',
 '이렇다',
 '이다',
 '어제',
 '매일',
 '아',
 '휴',
 '아이구',
 '아이쿠',
 '아이고',
 '어',
 '나',
 '우리',
 '저희',
 '따라',
 '의해',
 '을',
 '를',
 '에',
 '의',
 '가',
 '으로',
 '로',
 '에게',
 '뿐이다',
 '의거하여',
 '근거하여',
 '입각하여',
 '기준으로',
 '예하면',
 '예를 들면',
 '예를 들자면',
 '저',
 '소인',
 '소생',
 '저희',
 '지말고',
 '하지마',
 '하지마라',
 '다른',
 '물론',
 '또한',
 '그리고',
 '비길수 없다',
 '해서는 안된다',
 '뿐만 아니라',
 '만이 아니다',
 '만은 아니다',
 '막론하고',
 '관계없이',
 '그치지 않다',
 '그러나',
 '그런데',
 '하지만',
 '든간에',
 '논하지 않다',
 '따지지 않다',
 '설사',
 '비록',
 '더라도',
 '아니면',
 '만 못하다',
 '하는 편이 낫다',
 '불문하고',
 '향하여',
 '향해서',
 '향하다',
 '쪽으로',
 '틈타',
 '이용하여',
 '타다',
 '오르다',
 '제외하고',
 '이 외에',
 '이 밖에',
 '하여야',
 '비로소',
 '한다면 몰라도',
 '외에도',
 '이곳',
 '여기',
 '부터',
 '기점으로',
 '따라서',
 '할 생각이다',
 '하려고하다',
 '이리하여',
 '그리하여',
 '그렇게 함으로써',
 '하지만',
 '일때',
 '할때',
 '앞에서',
 '중에서',
 '보는데서',
 '으로써',
 '로써',
 '까지',
 '해야한다',
 '일것이다',
 '반드시',
 '할줄알다',
 '할수있다',
 '할수있어',
 '임에 틀림없다',
 '한다면',
 '등',
 '등등',
 '제',
 '겨우',
 '단지',
 '다만',
 '할뿐',
 '딩동',
 '댕그',
 '대해서',
 '대하여',
 '대하면',
 '훨씬',
 '얼

In [32]:
#제거되지 않은 불용어 추가
stopwords.extend(['는', '무단', '배포', '금지','고','email','protected','Copyright','NEWSIS','COM','기자','하는','게','뿐','다',''])

In [33]:
#제거되지 않은 불용어 추가
stopwords.extend(['익스','지난해','분기','통해','대한','올해','최근','지난','경우','때문','최대','대해','가장','현재','이후','라며','번가','이나','가운데','만원','프레','억원','관련','지금'])

In [34]:
# 특수 문자 및 매핑 테이블
punct = "/-'?!.,#$%\'()*+-/:;<=>@[\\]^_`{|}~" + '""“”’' + '∞θ÷α•à−β∅³π‘₹´°£€\×™√²—–&'
punct_mapping = {
    "‘": "'", "₹": "e", "´": "'", "°": "", "€": "e", "™": "tm", "√": " sqrt ", "×": "x", "²": "2",
    "—": "-", "–": "-", "’": "'", "_": "-", "`": "'", '“': '"', '”': '"', '“': '"', "£": "e",
    '∞': 'infinity', 'θ': 'theta', '÷': '/', 'α': 'alpha', '•': '.', 'à': 'a', '−': '-', 'β': 'beta',
    '∅': '', '³': '3', 'π': 'pi'
}

# HTML 태그 및 기타 전처리
def clean_str(text):
    text = re.sub('<[^>]*>', '', text)  # HTML 태그 제거
    text = re.sub('([a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+)', '', text)  # 이메일 제거
    text = re.sub('(http|ftp|https)://(?:[-\w.]|(?:%[\da-fA-F]{2}))+', '', text)  # URL 제거
    text = re.sub('([ㄱ-ㅎㅏ-ㅣ]+)', '', text)  # 한글 자음, 모음 제거
    text = re.sub('[^\w\s\n]', ' ', text)  # 특수기호 제거
    text = re.sub('[-=+,#/\?:^$.@*\"※~&%ㆍ!』\\‘|\(\)\[\]\<\>`\'…》]','', text)
    text = re.sub('\n', '.', text)
    # 숫자 제거
    text = re.sub(r'\d+', ' ', text)
    return text

# 특수 문자와 불용어 제거
def clean(text, punct, mapping):
    for p in mapping:
        text = text.replace(p, mapping[p])


    for p in punct:
        text = text.replace(p, f' {p} ')

    specials = {'\u200b': ' ', '…': ' ... ', '\ufeff': '', 'करना': '', 'है': ''}
    for s in specials:
        text = text.replace(s, specials[s])

    return text.strip()

# 불용어 제거
def remove_stopwords(tokens, stopwords):
    filtered_tokens = [word for word in tokens if word not in stopwords]
    return filtered_tokens

# 형태소 분석
def tokenize_and_tag(text):
    okt = Okt()
    tokens = okt.morphs(text)  # 형태소 분석 및 토큰화
    return tokens

# 종합 전처리 함수
def preprocess_text(text, punct, mapping, stopwords):
    text = clean_str(text)  # HTML 태그 및 기타 전처리
    text = clean(text, punct, mapping)  # 특수 문자와 불용어 제거
    tokens = tokenize_and_tag(text)  # 형태소 분석 및 토큰화
    tokens = remove_stopwords(tokens, stopwords)  # 불용어 제거
    return tokens


In [35]:
# 데이터프레임의 각 뉴스 본문에 대해 특수문자,숫자,공백제거,불용어제거,형태소 분석 및 토큰화
news['processed_text'] = news['raw_text'].apply(lambda x: preprocess_text(x, punct, punct_mapping, stopwords))

In [36]:
news.head()

,title,raw_text,date,url,processed_text
0,"""KC인증이 능사 아냐…계속 모니터링 해야""[中직구 안전성 논란④]","""KC인증이 능사 아냐…계속 모니터링 해야""[中직구 안전성 논란④] ""발암물질 및...",2024-06-03,https://www.newsis.com/view/?id=NISX20240531_0...,"[KC, 인증, 능사, 아냐, 계속, 모니터링, 해야, 中, 직구, 안전성, 논란,..."
1,"'직구 금지' 철회한 정부, 유해물품 안전성 검증할 방법은 - 머니S",최근 정부가 해외 직구 규제 대책을 발표했다가 번복하면서 소비자 혼란이 가중되고 있...,2024-06-03,https://www.moneys.co.kr/article/2024053117523...,"[정부, 해외, 직구, 규제, 대책, 발표, 했다가, 번복, 소비자, 혼란, 가중,..."
2,[테크왔숑] 늘어나는 해외직구에 신용카드 해킹 피해 급증···VPN으로 이중잠금,노드VPN의 다크 웹 모니터링·Double VPN 기능 통해 개인 정보 보호 철저 ...,2024-06-03,https://www.cnet.co.kr/view/?no=20240603113543,"[노드, VPN, 다크, 웹, 모니터링, Double, VPN, 기능, 개인, 정보..."
3,"짝퉁 골프, 중국계 플랫폼 중고 시장서 활개","가품 골프용품 거래 현장 [자료=클럽하우스] \n 알리, 테무, 쉬인 등 최근 급...",2024-06-04,http://jtbcgolf.joins.com/news/news_view.asp?n...,"[가품, 골프, 용품, 거래, 현장, 자료, 클럽, 하우스, ., 알리, 테무, 쉬..."
4,中 초저가 직구 급증…항공운임도 고공행진,中 초저가 직구 급증…항공운임도 고공행진 3·4분기 해상화물 몰려 더 뛸 듯 2일...,2024-06-02,https://www.hankyung.com/article/2024060299621,"[中, 초, 저가, 직구, 급증, 항공, 운임, 도, 고공, 행진, 해상, 화물, ..."


In [47]:
#단어 정규화-어간추출
# Okt 형태소 분석기 초기화
okt = Okt()

In [44]:
# 형태소 분석 및 어간 추출 또는 표제어 추출 함수 정의
def tokenize_and_lemmatize(text_list):
    if isinstance(text_list, list):  # 텍스트가 리스트인지 확인
        # 리스트의 각 요소를 문자열로 결합
        text = ' '.join(text_list)
        tokenized_text = okt.morphs(text)  # 형태소 분석
        # 여기서 어간 추출 또는 표제어 추출을 수행하고자 한다면 추가 작업을 수행합니다.
        # 예를 들어, 한글 형태소 분석기인 Okt를 사용하여 어간 추출을 수행할 수 있습니다.
        tokenized_text = [okt.morphs(word)[0] for word in tokenized_text]  # 어간 추출 예시
        lemmatized_text = ' '.join(tokenized_text)  # 토큰화된 텍스트를 다시 문자열로 결합
        return lemmatized_text
    else:
        return ""  # 텍스트가 리스트가 아닌 경우에는 빈 문자열을 반환

# 어간 추출 또는 표제어 추출 적용
news['processed_text'] = news['processed_text'].apply(tokenize_and_lemmatize)

# 결과 확인
print(news.head())

                                          title  \
0         "KC인증이 능사 아냐…계속 모니터링 해야"[中직구 안전성 논란④]   
1        '직구 금지' 철회한 정부, 유해물품 안전성 검증할 방법은 - 머니S   
2  [테크왔숑] 늘어나는 해외직구에 신용카드 해킹 피해 급증···VPN으로 이중잠금   
3                      짝퉁 골프, 중국계 플랫폼 중고 시장서 활개   
4                        中 초저가 직구 급증…항공운임도 고공행진   

                                            raw_text        date  \
0  "KC인증이 능사 아냐…계속 모니터링 해야"[中직구 안전성 논란④]  "발암물질 및...  2024-06-03   
1  최근 정부가 해외 직구 규제 대책을 발표했다가 번복하면서 소비자 혼란이 가중되고 있...  2024-06-03   
2  노드VPN의 다크 웹 모니터링·Double VPN 기능 통해 개인 정보 보호 철저 ...  2024-06-03   
3  가품 골프용품 거래 현장 [자료=클럽하우스]  \n 알리, 테무, 쉬인 등 최근 급...  2024-06-04   
4  中 초저가 직구 급증…항공운임도 고공행진  3·4분기 해상화물 몰려 더 뛸 듯 2일...  2024-06-02   

                                                 url  \
0  https://www.newsis.com/view/?id=NISX20240531_0...   
1  https://www.moneys.co.kr/article/2024053117523...   
2     https://www.cnet.co.kr/view/?no=20240603113543   
3  http://jtbcgolf.joins.com/news/news_view.asp?n...   
4   

In [48]:
news.head()

,title,raw_text,date,url,processed_text
0,"""KC인증이 능사 아냐…계속 모니터링 해야""[中직구 안전성 논란④]","""KC인증이 능사 아냐…계속 모니터링 해야""[中직구 안전성 논란④] ""발암물질 및...",2024-06-03,https://www.newsis.com/view/?id=NISX20240531_0...,KC 인증 능사 아냐 계속 모니터링 해야 中 직구 안전성 논란 ④ 발암 물질 유해 ...
1,"'직구 금지' 철회한 정부, 유해물품 안전성 검증할 방법은 - 머니S",최근 정부가 해외 직구 규제 대책을 발표했다가 번복하면서 소비자 혼란이 가중되고 있...,2024-06-03,https://www.moneys.co.kr/article/2024053117523...,정부 해외 직구 규제 대책 발표 했다가 번복 소비자 혼란 가중 되고 해외 직구 국가...
2,[테크왔숑] 늘어나는 해외직구에 신용카드 해킹 피해 급증···VPN으로 이중잠금,노드VPN의 다크 웹 모니터링·Double VPN 기능 통해 개인 정보 보호 철저 ...,2024-06-03,https://www.cnet.co.kr/view/?no=20240603113543,노드 VPN 다크 웹 모니터링 Double VPN 기능 개인 정보 보호 철저 씨넷 ...
3,"짝퉁 골프, 중국계 플랫폼 중고 시장서 활개","가품 골프용품 거래 현장 [자료=클럽하우스] \n 알리, 테무, 쉬인 등 최근 급...",2024-06-04,http://jtbcgolf.joins.com/news/news_view.asp?n...,가품 골프 용품 거래 현장 자료 클럽 하우스 . 알리 테무 쉬 인 급성 장 한 중국...
4,中 초저가 직구 급증…항공운임도 고공행진,中 초저가 직구 급증…항공운임도 고공행진 3·4분기 해상화물 몰려 더 뛸 듯 2일...,2024-06-02,https://www.hankyung.com/article/2024060299621,中 초 저가 직구 급증 항공 운임 도 고공 행진 해상 화물 몰려 더 뛸 듯 항공 업...


In [49]:
news_noun=news.copy()

In [50]:
#명사만 추출
# Okt 형태소 분석기를 초기화합니다.
okt = Okt()

# 명사만 추출하는 함수 정의
def extract_nouns(text):
    nouns = okt.nouns(text)
    return ' '.join(nouns)

# 'processed_text' 칼럼에서 명사만 추출하여 새로운 칼럼 'nouns_only'에 저장
news_noun['nouns_only'] = news_noun['processed_text'].apply(extract_nouns)
news_noun.head()
# for text_list in news['processed_text']:  # 모든 기사에 대해
    # nouns = okt.nouns(text_list)  # 명사만 추출

,title,raw_text,date,url,processed_text,nouns_only
0,"""KC인증이 능사 아냐…계속 모니터링 해야""[中직구 안전성 논란④]","""KC인증이 능사 아냐…계속 모니터링 해야""[中직구 안전성 논란④] ""발암물질 및...",2024-06-03,https://www.newsis.com/view/?id=NISX20240531_0...,KC 인증 능사 아냐 계속 모니터링 해야 中 직구 안전성 논란 ④ 발암 물질 유해 ...,인증 능사 계속 모니터링 직구 안전성 논란 발암 물질 유해 물질 검 문제 반복 발생...
1,"'직구 금지' 철회한 정부, 유해물품 안전성 검증할 방법은 - 머니S",최근 정부가 해외 직구 규제 대책을 발표했다가 번복하면서 소비자 혼란이 가중되고 있...,2024-06-03,https://www.moneys.co.kr/article/2024053117523...,정부 해외 직구 규제 대책 발표 했다가 번복 소비자 혼란 가중 되고 해외 직구 국가...,정부 해외 직구 규제 대책 발표 번복 소비자 혼란 가중 해외 직구 국가 통합인증 마...
2,[테크왔숑] 늘어나는 해외직구에 신용카드 해킹 피해 급증···VPN으로 이중잠금,노드VPN의 다크 웹 모니터링·Double VPN 기능 통해 개인 정보 보호 철저 ...,2024-06-03,https://www.cnet.co.kr/view/?no=20240603113543,노드 VPN 다크 웹 모니터링 Double VPN 기능 개인 정보 보호 철저 씨넷 ...,노드 다크 웹 모니터링 기능 개인 정보 보호 철저 씨넷 코리아 황진영 금융감독원 피...
3,"짝퉁 골프, 중국계 플랫폼 중고 시장서 활개","가품 골프용품 거래 현장 [자료=클럽하우스] \n 알리, 테무, 쉬인 등 최근 급...",2024-06-04,http://jtbcgolf.joins.com/news/news_view.asp?n...,가품 골프 용품 거래 현장 자료 클럽 하우스 . 알리 테무 쉬 인 급성 장 한 중국...,가품 골프 용품 거래 현장 자료 클럽 하우스 알리 테무 쉬 인 급성 장 중국 계 유...
4,中 초저가 직구 급증…항공운임도 고공행진,中 초저가 직구 급증…항공운임도 고공행진 3·4분기 해상화물 몰려 더 뛸 듯 2일...,2024-06-02,https://www.hankyung.com/article/2024060299621,中 초 저가 직구 급증 항공 운임 도 고공 행진 해상 화물 몰려 더 뛸 듯 항공 업...,초 저가 직구 급증 항공 운임 도 고공 행진 해상 화물 더 듯 항공 업계 말 기준 ...


In [51]:
def filter_nouns(text):
    nouns = text.split()
    filtered_nouns = [noun for noun in nouns if len(noun) > 1]
    return ' '.join(filtered_nouns)

# 'processed_text' 칼럼에서 명사만 추출하여 새로운 칼럼 'filtered_nouns_only'에 저장
news_noun['filtered_nouns_only'] = news_noun['nouns_only'].apply(filter_nouns)

In [52]:
news_noun.head()

,title,raw_text,date,url,processed_text,nouns_only,filtered_nouns_only
0,"""KC인증이 능사 아냐…계속 모니터링 해야""[中직구 안전성 논란④]","""KC인증이 능사 아냐…계속 모니터링 해야""[中직구 안전성 논란④] ""발암물질 및...",2024-06-03,https://www.newsis.com/view/?id=NISX20240531_0...,KC 인증 능사 아냐 계속 모니터링 해야 中 직구 안전성 논란 ④ 발암 물질 유해 ...,인증 능사 계속 모니터링 직구 안전성 논란 발암 물질 유해 물질 검 문제 반복 발생...,인증 능사 계속 모니터링 직구 안전성 논란 발암 물질 유해 물질 문제 반복 발생 사...
1,"'직구 금지' 철회한 정부, 유해물품 안전성 검증할 방법은 - 머니S",최근 정부가 해외 직구 규제 대책을 발표했다가 번복하면서 소비자 혼란이 가중되고 있...,2024-06-03,https://www.moneys.co.kr/article/2024053117523...,정부 해외 직구 규제 대책 발표 했다가 번복 소비자 혼란 가중 되고 해외 직구 국가...,정부 해외 직구 규제 대책 발표 번복 소비자 혼란 가중 해외 직구 국가 통합인증 마...,정부 해외 직구 규제 대책 발표 번복 소비자 혼란 가중 해외 직구 국가 통합인증 마...
2,[테크왔숑] 늘어나는 해외직구에 신용카드 해킹 피해 급증···VPN으로 이중잠금,노드VPN의 다크 웹 모니터링·Double VPN 기능 통해 개인 정보 보호 철저 ...,2024-06-03,https://www.cnet.co.kr/view/?no=20240603113543,노드 VPN 다크 웹 모니터링 Double VPN 기능 개인 정보 보호 철저 씨넷 ...,노드 다크 웹 모니터링 기능 개인 정보 보호 철저 씨넷 코리아 황진영 금융감독원 피...,노드 다크 모니터링 기능 개인 정보 보호 철저 씨넷 코리아 황진영 금융감독원 피싱 ...
3,"짝퉁 골프, 중국계 플랫폼 중고 시장서 활개","가품 골프용품 거래 현장 [자료=클럽하우스] \n 알리, 테무, 쉬인 등 최근 급...",2024-06-04,http://jtbcgolf.joins.com/news/news_view.asp?n...,가품 골프 용품 거래 현장 자료 클럽 하우스 . 알리 테무 쉬 인 급성 장 한 중국...,가품 골프 용품 거래 현장 자료 클럽 하우스 알리 테무 쉬 인 급성 장 중국 계 유...,가품 골프 용품 거래 현장 자료 클럽 하우스 알리 테무 급성 중국 유통 플랫폼 중고...
4,中 초저가 직구 급증…항공운임도 고공행진,中 초저가 직구 급증…항공운임도 고공행진 3·4분기 해상화물 몰려 더 뛸 듯 2일...,2024-06-02,https://www.hankyung.com/article/2024060299621,中 초 저가 직구 급증 항공 운임 도 고공 행진 해상 화물 몰려 더 뛸 듯 항공 업...,초 저가 직구 급증 항공 운임 도 고공 행진 해상 화물 더 듯 항공 업계 말 기준 ...,저가 직구 급증 항공 운임 고공 행진 해상 화물 항공 업계 기준 홍콩 북미 노선 화...


In [ ]:
# 'filtered_nouns_only' 열에서 모든 명사의 개수를 세는 함수 정의
def count_total_nouns(column):
    total_nouns = column.str.split().apply(len).sum()
    return total_nouns

# 전체 데이터프레임에서 'filtered_nouns_only' 열의 명사 개수를 셉니다.
total_noun_count = count_total_nouns(news_noun['filtered_nouns_only'])
total_noun_count_2 = count_total_nouns(news_noun['nouns_only'])

In [53]:
# 하나라도 NaN 값을 가지는 행을 뽑아냅니다.
rows_with_nan = news_noun[news_noun.isna().any(axis=1)]

# 결과를 출력합니다.
print(rows_with_nan)

Empty DataFrame
Columns: [title, raw_text, date, url, processed_text, nouns_only, filtered_nouns_only]
Index: []


2019년 이후의 데이터만 사용하기 위해 이전년도 데이터 삭제

In [55]:
news_noun.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4743 entries, 0 to 4742
Data columns (total 7 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   title                4743 non-null   object
 1   raw_text             4743 non-null   object
 2   date                 4743 non-null   object
 3   url                  4743 non-null   object
 4   processed_text       4743 non-null   object
 5   nouns_only           4743 non-null   object
 6   filtered_nouns_only  4743 non-null   object
dtypes: object(7)
memory usage: 259.5+ KB


In [60]:
news_noun['date']

0       2024-06-03
1       2024-06-03
2       2024-06-03
3       2024-06-04
4       2024-06-02
           ...    
4738    2024-05-09
4739    2024-05-09
4740    2024-04-28
4741    2024-03-11
4742    2023-05-02
Name: date, Length: 4743, dtype: object

In [61]:
news_noun['date']=pd.to_datetime(news_noun.date)

In [62]:
news_noun['date'].sort_values(ascending=False)

350    2024-06-04
3574   2024-06-04
3417   2024-06-04
3428   2024-06-04
3437   2024-06-04
          ...    
3720   2008-07-16
3848   2006-01-04
2494   2006-01-04
2675   2006-01-04
547    2006-01-04
Name: date, Length: 4743, dtype: datetime64[ns]

In [63]:
news_noun['date'].dt.year.unique()

array([2024, 2020, 2006, 2023, 2022, 2021, 2008, 2019], dtype=int32)

In [64]:
news_noun['date'].dt.year.value_counts()

date
2024    3831
2023     638
2022     100
2020      83
2021      79
2019       7
2006       4
2008       1
Name: count, dtype: int64

In [66]:
ans=news_noun.loc[news_noun.date.dt.year.isin([2006,2008])].index
ans

Index([547, 2494, 2675, 3720, 3848], dtype='int64')

In [67]:
news_noun=news_noun.drop(ans)

In [72]:
news_noun['date'].dt.year.unique() #2019이전의 데이터가 삭제된 것을 확인

array([2024, 2020, 2023, 2022, 2021, 2019], dtype=int32)

In [ ]:
# 데이터 전처리 완료 후 데이터 저장
# Google Drive에 마운트된 경로 설정
drive_path = '/content/drive/MyDrive/관세청공모전/크롤링데이터'  # 실제 경로로 변경
cleaned_csv_path = os.path.join(drive_path, 'news_noun.csv')
news_noun.to_csv(cleaned_csv_path, index=False)